# Introduction to Nibiru SDK

The python sdk allows you to create queries and send transactions to a Nibiru chain.
It allows to interact with all the modules and can be leveraged to automate trading strategies or monitor them.
This notebook will guide you on opening and closing positions.

### 1. Install Nibiru package

In [ ]:
!pip install nibiru

### 2. Prepare wallet mnemonic key (or private key)
If you don't have a mnemonic key, generate it using some online service or run the code below:

In [ ]:
from nibiru import PrivateKey
mnemonic, private_key = PrivateKey.generate()
print(mnemonic)

### 3. Get test tokens from the nibiru faucet
Faucet URL: [https://app.nibiru.fi/faucet](https://app.nibiru.fi/faucet)

### 4. Create nibiru chain client and authenticate using mnemonic
The client below uses ITN-2 Nibiru testnet chain.
Use your mnemonic key or private key to authenticate.
If you took money from the faucet, your balances should not be empty.

In [ ]:
import json
from nibiru import Network, ChainClient, Msg, Coin

mnemonic = "put your mnemonic here"
client = ChainClient(network=Network.testnet(2))
client.authenticate(mnemonic=mnemonic)  # alternatively use private_key_hex = your_key_hex

# Check balance
print(json.dumps(client.query.get_bank_balances(client.address)["balances"], indent=2))

### 5. Create a tx to open perp trading position
Position is: LONG (=BUY),

pair = ubtc:unusd: prefix `u` means `μ` or micro. So `unusd = micro unusd = 1/1000_000 nusd`, same for ubtc.

margin = 10: (value in quote amount which is unusd in our case)

leverage = 2: your position notional will be 2 * 10

In [ ]:
pair = "ubtc:unusd"

output = client.tx.execute_msgs(
    Msg.perp.open_position(
        pair=pair,
        is_long=True,
        margin=10,
        leverage=2,
    )
)
print(output)

If your TX has succeeded, you should get output something like:
```
ExecuteTxResp(code=0, tx_hash='946A4252423A0AC183C1BAA2B76CCB25512DC9A18861D8782EFE7F14512CBDF6', log='[]')
```
Query your TX hash to get TX details:

In [ ]:
output = client.query.tx_by_hash("707F725DA2A04FD865FA6FEAF431DF89459F6A5C076B5AF93C627979C9B6CDF1")
print(output)

Or directly query the state of your position:

In [ ]:
position = client.query.perp.position(pair, client.address)
print(json.dumps(position, indent=4))

We can then add margin to our position:

In [ ]:
output = client.tx.execute_msgs(
    Msg.perp.add_margin(
        pair=pair,
        margin=Coin(1, "unusd"),
    )
)
print(output)

Or remove margin:

In [ ]:
output = client.tx.execute_msgs(
    Msg.perp.remove_margin(
        pair=pair,
        margin=Coin(1, "unusd")
    )
)
print(output)

We can then totally close the position:

In [ ]:
output = client.tx.execute_msgs(
    Msg.perp.close_position(
        pair=pair,
    )
)
print(output)

## Multi-transactions
We can build multiple messages into a single transaction to be sure that they are executed consecutively.

It can be useful for example to send tokens after removing margins from a position.

Lets create a transction with:
- Agent open a position
- Agent add margin
- Agent close the position

In [ ]:
output = client.tx.execute_msgs(
    [
        Msg.perp.open_position(
            pair=pair,
            is_long=True,
            margin=10,
            leverage=1,
        ),
        Msg.perp.add_margin(
            pair=pair,
            margin=Coin(1, "unusd"),
        ),
        Msg.perp.close_position(
            pair=pair,
        ),
    ]
)
print(output)

In [ ]:
position = client.query.perp.all_positions(client.address)
print(json.dumps(position, indent=4))

That's it! Good luck and happy Nibiru coding!